<a href="https://colab.research.google.com/github/wallas999/Mis_Proyectos/blob/main/Elevators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Versión 1.1
# 1 elevador, 1 lógica de operación
# Subida a GitHub el 8 de agosto de 2025


#* La función con algoritmo SCAN clásico
#* Cálculo de energía simplificado
#* Visualización del historial con pandas


# Importamos pandas para mostrar resultados ordenados
import pandas as pd
from IPython.display import display, HTML

def simular_scan(
    llamadas,
    piso_inicial,
    consumo_subida=4.0,     # Wh por piso subido
    consumo_bajada=1.0,     # Wh por piso bajado
    consumo_parada=1.0,     # Wh por parada
    tiempo_por_piso=5,      # segundos por piso recorrido
    tiempo_parada=10        # segundos por parada
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    llamadas_arriba = sorted([p for p in llamadas if p > piso_actual])
    llamadas_abajo = sorted([p for p in llamadas if p < piso_actual], reverse=True)

    # Atender llamadas hacia arriba
    for piso_destino in llamadas_arriba:
        pisos = abs(piso_destino - piso_actual)
        energia_mov = pisos * consumo_subida
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa

        tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual,
            "Hasta": piso_destino,
            "Sube": True,
            "Distancia (pisos)": pisos,
            "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total,
            "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = piso_destino

    # Atender llamadas hacia abajo
    for piso_destino in llamadas_abajo:
        pisos = abs(piso_destino - piso_actual)
        energia_mov = pisos * consumo_bajada
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa

        tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual,
            "Hasta": piso_destino,
            "Sube": False,
            "Distancia (pisos)": pisos,
            "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total,
            "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = piso_destino

    return pd.DataFrame(historial), consumo_total, tiempo_total





In [21]:
# Parámetros de prueba
piso_inicial = 3
llamadas = [2, 8, 4, 6]

# Ejecutar simulación
df_historial, energia_total, tiempo_total = simular_scan(llamadas, piso_inicial)

# Crear copia para modificar nombres de columnas
df_mod = df_historial.copy()

# Cambiar nombres para mostrar saltos de línea con <br>
df_mod.columns = [
    "Desde",
    "Hasta",
    "Sube",
    "Distancia<br>(pisos)",
    "Energía<br>etapa (Wh)",
    "Energía<br>acumulada (Wh)",
    "Tiempo<br>etapa (s)",
    "Tiempo<br>acumulado (s)"
]

# Mostrar DataFrame con encabezados en dos líneas
display(HTML(df_mod.to_html(escape=False)))

print(f"\nEnergía total: {energia_total:.2f} Wh")
print(f"Tiempo total: {tiempo_total} segundos")


,Desde,Hasta,Sube,Distancia(pisos),Energíaetapa (Wh),Energíaacumulada (Wh),Tiempoetapa (s),Tiempoacumulado (s)
0,3,4,True,1,5.0,5.0,15,15
1,4,6,True,2,9.0,14.0,20,35
2,6,8,True,2,9.0,23.0,20,55
3,8,2,False,6,7.0,30.0,40,95



Energía total: 30.00 Wh
Tiempo total: 95 segundos


In [22]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Extraer lista de pisos por donde pasa el ascensor, empezando por el piso inicial
pisos_recorrido = [piso_inicial] + df_historial["Hasta"].tolist()

plt.ioff() #evita que se cree una segunda figura (estática) del edificio
fig, ax = plt.subplots(figsize=(2.5, 4))

# Configurar eje Y con pisos (invertido para que el piso 1 esté abajo)
ax.set_ylim(min(pisos_recorrido)-1, max(pisos_recorrido)+1)
ax.set_xlim(0, 1)
ax.set_xticks([]) # se oculta la graduación del eje X
ax.set_ylabel("Piso")
ax.set_title("Simulación Ascensor \n (Algoritmo SCAN)")

# Dibujar líneas horizontales para cada piso
for piso in range(min(pisos_recorrido), max(pisos_recorrido)+1):
    ax.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')

# Punto que representa el ascensor
punto_ascensor, = ax.plot([], [], 's', markersize=20, color='blue')

# Texto para mostrar energía consumida en el paso
texto_energia = ax.text(0.5, 0.5-0.5, '', ha='center')

def init():
    punto_ascensor.set_data([0.5], [pisos_recorrido[0]])
    texto_energia.set_text('')
    return punto_ascensor, texto_energia

def actualizar(frame):
    piso = pisos_recorrido[frame]
    punto_ascensor.set_data([0.5], [piso])


    if frame == 0:
        texto_energia.set_text(f"Piso inicial: {piso}")
    else:
        energia = df_historial.iloc[frame-1]["Energía etapa (Wh)"]
        texto_energia.set_text(f"Energía etapa: \n {energia:.1f} Wh")

    return punto_ascensor, texto_energia

from IPython.display import HTML
from IPython.display import clear_output, HTML


ani = animation.FuncAnimation(
    fig, actualizar, frames=len(pisos_recorrido),
    init_func=init, blit=True, interval=1000, repeat=False
)

HTML(ani.to_jshtml())  # Opción 1: usar JavaScript (más liviano)

